In [ ]:
# cleaner for arxiv papers
def arxiv_cleaner(dir,limit):
    import nltk.corpus  
    from nltk import word_tokenize
    from nltk.corpus import stopwords
    import enchant
    import os
    
    stop_words = set(stopwords.words('english'))
    d = enchant.Dict("en_US")
    
    stop=0
    counter=0
    for root,dirs,files, in os.walk(dir):
        for file in files:
            stop+=1
            if stop == limit:
                break
            path_to_txt = os.path.join(root, file)
            [stem, ext] = os.path.splitext(path_to_txt)
            if ext == '.txt':
                paper = open(stem+ext,'r')
                paper_token=word_tokenize(paper.read()) 
                #correction of words broken into two parts
                words_c = paper_token
                words_cleaned = []
                wrong=[]
                n=0
                counter+=1

                for w in range(len(words_c)):
                    n+=1
                    if 'nnn' in words_c[w]:
                        j=words_c[w][words_c[w].find('nn')+2:len(words_c[w])]
                        k=j[1:]
                        l=words_c[w][:words_c[w].find('nn')+1]
                        if len(j) > 1 and d.check(j) == True:
                            words_cleaned.append(j)
                        if len(k) > 2 and d.check(k) == True:
                            words_cleaned.append(k)
                        if len(l) > 1 and d.check(l) == True:
                            words_cleaned.append(l)
                    elif 'nn' in words_c[w]:
                        if words_c[w].startswith('nn'):
                            clean=words_c[w].replace('nn','')
                            clean1=words_c[w-1]+clean
                            if d.check(clean1) == True:
                                words_cleaned.append(clean1)
                            elif len(clean) > 1 and d.check(clean) == True:
                                words_cleaned.append(clean)
                            else:
                                continue
                        elif words_c[w].endswith('nn'):
                            clean=words_c[w].replace('nn','')
                            if n < len(words_c)-1:
                                clean1=clean+words_c[w+1]
                            else:
                                clean1=clean+words_c[w]
                            if d.check(clean1) == True:
                                words_cleaned.append(clean1)
                            elif len(clean) > 1 and d.check(clean) == True:
                                words_cleaned.append(clean)
                            else:
                                wrong.append(words_c[w])
                        else:
                            j=words_c[w][words_c[w].find('nn')+1:len(words_c[w])]
                            k=j[1:]
                            l=words_c[w][:words_c[w].find('nn')]
                            if len(j) > 1 and d.check(j) == True:
                                words_cleaned.append(j)
                            if len(k) > 2 and d.check(k) == True:
                                words_cleaned.append(k)
                            if len(l) > 1 and d.check(l) == True:
                                words_cleaned.append(l)
                    elif (len(words_c[w]) > 2) and (d.check(words_c[w]) == True):
                        words_cleaned.append(words_c[w])
                    elif len(words_c[w]) > 2:
                        words_cleaned.append(words_c[w])
                    elif words_c[w] == "em":
                        words_cleaned.append(words_c[w])
                    else:    
                        wrong.append(words_c[w])
                        continue

                #removing stopwords
                words_cleaned_1 = [w for w in words_cleaned if not w in stop_words]

                #cleaning from nonsense 'x'-expressions (equations)
                words_cleaned_2 = [w for w in words_cleaned_1 if not w.startswith('x') and not w.endswith('x') and len(w) > 2]

                path_to_cleaned=stem+'_cleaned'+ext
                with open(path_to_cleaned, 'w') as cleaned:
                    #print("Writing contents to " + path_to_cleaned)
                    print('paper #',counter)
                    cleaned.write(str(words_cleaned_2))